In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [2]:
enade_df = pd.read_csv('enade_merge_08-11-14-17.csv',sep=',',decimal='.')

In [3]:
enade_df

,ano,curso,nome_curso,uf,regiao,codigo_IES,categoria_adm_IES,organizacao_academica,turno,nota_geral,...,moradia,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado
0,2008,82786,RC,SP,SE,1345,IES privada,Faculdade,Noturno,31.1,...,Cônjuge/filhos,Nenhuma,EM tradicional,Algumas,Algumas,Nenhum,Regular,31,13,1
1,2008,61178,RC,SP,SE,1743,IES privada,Faculdade,Noturno,32.2,...,Habitação individual/coletiva,Nenhuma,EM tradicional,Maior parte,Maior parte,Maior parte,Boa,34,15,3
2,2008,66681,RC,ES,SE,1808,IES pública,C.F.E.T,Noturno,32.6,...,Parentes,Três,EM tradicional,Todas,Todas,Todos aspectos,Muito boa,19,1,4
3,2008,66681,RC,ES,SE,1808,IES pública,C.F.E.T,Integral,24.3,...,Universidade,Nenhuma,EM tradicional,Todas,Maior parte,Somente alguns,Boa,31,12,3
4,2008,71315,RC,SP,SE,1742,IES privada,Faculdade,Noturno,19.1,...,Parentes,Uma,EM tradicional,Maior parte,Maior parte,Maior parte,Boa,20,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123487,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,46.8,...,Parentes,Quatro,EM tradicional,Todas,Todas,Maior parte,Muito boa,18,1,2
123488,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,26.9,...,Sozinho,Nenhuma,EM tradicional,Maior parte,Maior parte,Somente alguns,Regular,23,5,2
123489,2017,5001211,ADS,SC,S,3295,IES privada,Faculdade,Diurno,20.5,...,Parentes,Três,EM tradicional,Todas,Todas,Somente alguns,Regular,34,9,1
123490,2017,5001284,LCC,SC,S,494,IES privada,Universidade,Integral,52.0,...,Parentes,Três,EM tradicional,Nenhuma,Todas,Todos aspectos,Muito boa,23,6,2


In [4]:
#achando valores para kflod cross validation
for i in range(1,123493):
    if 123492%i ==0 :
        print(i)

1
2
3
4
6
12
41
82
123
164
246
251
492
502
753
1004
1506
3012
10291
20582
30873
41164
61746
123492


### Normalização

https://medium.com/tentando-ser-um-unic%C3%B3rnio/porqu%C3%AA-e-quando-%C3%A9-necess%C3%A1rio-normalizar-os-dados-92e5cce445aa

In [5]:
coluna_nt_geral = enade_df[['nota_geral']]

In [6]:
notas = ['nota_geral', 'nota_formacao_geral', 'nota_obj_formacao_geral', 'nota_dis_formacao_geral', 'nota_componente_especifico', 'nota_obj_componente_especifico',
        'nota_dis_componente_especifico']

In [7]:
cursos = list(enade_df['nome_curso'].unique())
cursos

['RC', 'SI', 'ADS', 'EC', 'BCC', 'LCC', 'GTI']

In [8]:
anos = list(enade_df['ano'].unique())
anos

[2008, 2011, 2014, 2017]

In [9]:
#normalizacao min max para cada curso, em cada ano
for i in anos:
    for j in cursos:
        df = enade_df['nota_geral'].loc[(enade_df.ano == i) & (enade_df.nome_curso == j)]
        df = ((df - df.min())/(df.max() - df.min()))
        enade_df['nota_geral'].loc[(enade_df.ano == i) & (enade_df.nome_curso == j)] = df

/home/luiscapelari/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/luiscapelari/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/luiscapelari/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [10]:
enade_df.groupby(['nome_curso','ano'])['nota_geral'].describe()

count      mean       std  min       25%       50%  \
nome_curso ano                                                          
ADS        2008   2223.0  0.500879  0.144432  0.0  0.399760  0.498199   
           2011   7034.0  0.459106  0.174775  0.0  0.332955  0.444318   
           2014  11107.0  0.442018  0.144627  0.0  0.339891  0.433880   
           2017   9352.0  0.443363  0.141610  0.0  0.343267  0.437086   
BCC        2008   2761.0  0.422704  0.147542  0.0  0.318182  0.400932   
           2011   9458.0  0.399784  0.154302  0.0  0.295261  0.386391   
           2014   8263.0  0.462295  0.142893  0.0  0.362123  0.454735   
           2017   8103.0  0.454610  0.142316  0.0  0.355415  0.443743   
EC         2008    243.0  0.458555  0.193295  0.0  0.320312  0.433239   
           2011   1899.0  0.425278  0.141742  0.0  0.333333  0.416378   
           2014   2542.0  0.490625  0.164769  0.0  0.388060  0.495736   
           2017   3280.0  0.471829  0.145649  0.0  0.370062  0.460499   
GTI        2017   3104.0  0.518109  0.152168  0.0  0.413440  0.519362   
LCC        2011    679.0  0.482987  0.154452  0.0  0.373333  0.472593   
           2014   1778.0  0.567280  0.169553  0.0  0.448167  0.577320   
           2017   1057.0  0.511576  0.165340  0.0  0.390704  0.512563   
RC         2008   1705.0  0.445161  0.151482  0.0  0.341549  0.427230   
           2011   3300.0  0.413261  0.135518  0.0  0.321547  0.404420   
           2014   3849.0  0.460909  0.144321  0.0  0.358306  0.452769   
           2017   2397.0  0.441078  0.151681  0.0  0.329843  0.433246   
SI         2008   4598.0  0.471462  0.134617  0.0  0.381844  0.466859   
           2011   9975.0  0.403217  0.134662  0.0  0.311081  0.400534   
           2014  13238.0  0.493303  0.145242  0.0  0.389140  0.489819   
           2017  11547.0  0.511499  0.150083  0.0  0.406644  0.509737   

                      75%  max  
nome_curso ano                  
ADS        2008  0.597839  1.0  
           2011  0.577273  1.0  
           2014  0.537705  1.0  
           2017  0.539735  1.0  
BCC        2008  0.501166  1.0  
           2011  0.493317  1.0  
           2014  0.556712  1.0  
           2017  0.550999  1.0  
EC         2008  0.553267  1.0  
           2011  0.508651  1.0  
           2014  0.600213  1.0  
           2017  0.566528  1.0  
GTI        2017  0.623007  1.0  
LCC        2011  0.579259  1.0  
           2014  0.690722  1.0  
           2017  0.628141  1.0  
RC         2008  0.538732  1.0  
           2011  0.498343  1.0  
           2014  0.554832  1.0  
           2017  0.540576  1.0  
SI         2008  0.559078  1.0  
           2011  0.489987  1.0  
           2014  0.596154  1.0  
           2017  0.615120  1.0

### Colunas removidas

As colunas removidas foram
- codigo_IES
- formatura
- inicio graduacao
- idade
- ano
- curso 

In [11]:
colunas_categoricas = ['nome_curso','uf', 'regiao', 'categoria_adm_IES', 'organizacao_academica', 'turno',  
                       'tipo_ensino_medio', 'range_idade', 'sexo', 'cor_raca', 'escolaridade_pai', 'escolaridade_mae', 
                       'renda_familiar', 'cotas', 'recebeu_cota',  'trabalha', 'livros_ano',
                       'horas_estudo', 'estado_civil', 'moradia', 'pessoas_moradia', 'ensino_medio', 'condicoes_salas_estudantes',
                       'condicoes_praticas_materiais', 'plano_ensino', 'avaliacao_curso', 'idade_comeco_graduacao', 'tempo_ocioso', 'tempo_cursado']

In [12]:
copia_df = enade_df.copy()

In [13]:
# deixar o dataset apenas com colunas que podem passar por one-hot encode

colunas_df = list(enade_df.columns)
for i in colunas_df:
    if i not in colunas_categoricas:
        enade_df.drop(columns=i, inplace=True)

### Label encoder
transformar categorias em variavel númérica

In [14]:
translate_df = {}

In [15]:
#categorizacao da nota geral
bins = [0.0, 20.0, 40.0, 60.0, 80.0, 100.0]
labels = ['Muito baixa', 'Baixa', 'Média', 'Alta', 'Muito alta']
enade_df['nt_geral_categoria'] = pd.cut(coluna_nt_geral['nota_geral'], bins, labels = labels,include_lowest = True)

In [16]:
colunas_categoricas.append('nt_geral_categoria')

In [17]:
label_encoder = LabelEncoder()

#transformando categoria de cada coluna em numeros
for i in colunas_categoricas:
    enade_df[i] = label_encoder.fit_transform(enade_df[i])
    translate_df[i] = list(label_encoder.classes_)

In [18]:
enade_df

,nome_curso,uf,regiao,categoria_adm_IES,organizacao_academica,turno,tipo_ensino_medio,range_idade,sexo,cor_raca,...,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado,nt_geral_categoria
0,5,25,4,0,2,2,2,2,0,1,...,3,1,0,0,1,5,17,13,1,1
1,5,25,4,0,2,2,2,2,1,1,...,3,1,1,1,0,0,20,15,3,1
2,5,7,4,1,0,2,2,0,0,1,...,6,1,4,4,4,2,5,1,4,1
3,5,7,4,1,0,1,2,2,0,3,...,3,1,4,1,3,0,17,12,3,1
4,5,25,4,0,2,2,2,0,1,3,...,7,1,1,1,0,0,6,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123487,0,23,3,0,2,0,2,0,1,0,...,4,1,4,4,0,2,4,1,2,4
123488,0,23,3,0,2,0,1,0,1,1,...,3,1,1,1,3,5,9,5,2,1
123489,0,23,3,0,2,0,2,2,1,1,...,6,1,4,4,3,5,20,9,1,1
123490,4,23,3,0,5,1,2,0,1,1,...,6,1,2,4,4,2,9,6,2,4


In [19]:
enade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123492 entries, 0 to 123491
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   nome_curso                    123492 non-null  int64
 1   uf                            123492 non-null  int64
 2   regiao                        123492 non-null  int64
 3   categoria_adm_IES             123492 non-null  int64
 4   organizacao_academica         123492 non-null  int64
 5   turno                         123492 non-null  int64
 6   tipo_ensino_medio             123492 non-null  int64
 7   range_idade                   123492 non-null  int64
 8   sexo                          123492 non-null  int64
 9   cor_raca                      123492 non-null  int64
 10  escolaridade_pai              123492 non-null  int64
 11  escolaridade_mae              123492 non-null  int64
 12  renda_familiar                123492 non-null  int64
 13  cotas         

In [20]:
enade_df.describe()

,nome_curso,uf,regiao,categoria_adm_IES,organizacao_academica,turno,tipo_ensino_medio,range_idade,sexo,cor_raca,...,pessoas_moradia,ensino_medio,condicoes_salas_estudantes,condicoes_praticas_materiais,plano_ensino,avaliacao_curso,idade_comeco_graduacao,tempo_ocioso,tempo_cursado,nt_geral_categoria
count,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,...,123492.000000,123492.000000,123492.00000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000,123492.000000
mean,2.917468,17.162116,2.994639,0.281856,3.278690,1.599553,1.670230,0.650674,0.845286,1.756956,...,4.079633,1.495975,2.33969,2.192312,2.061559,1.946490,8.789063,4.542019,3.563259,2.206718
std,2.531909,7.402983,1.284171,0.449906,1.663014,0.716784,0.474708,0.878932,0.361634,1.109934,...,2.394630,1.164492,1.61080,1.613268,1.878284,1.790233,6.042916,4.987957,1.854933,1.542038
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,10.000000,2.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,5.000000,1.000000,2.000000,1.000000
50%,2.000000,18.000000,4.000000,0.000000,3.000000,2.000000,2.000000,0.000000,1.000000,1.000000,...,4.000000,1.000000,2.00000,1.000000,3.000000,2.000000,7.000000,3.000000,3.000000,1.000000
75%,6.000000,25.000000,4.000000,1.000000,5.000000,2.000000,2.000000,1.000000,1.000000,3.000000,...,6.000000,1.000000,4.00000,4.000000,4.000000,2.000000,11.000000,6.000000,4.000000,4.000000
max,6.000000,26.000000,4.000000,1.000000,5.000000,2.000000,2.000000,4.000000,1.000000,4.000000,...,7.000000,4.000000,4.00000,4.000000,4.000000,5.000000,53.000000,46.000000,23.000000,4.000000


### One Hot Encoding
transformar variaveis núméricas em colunas de valores binarios

https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/

https://arthurlambletvaz.medium.com/one-hot-encoding-o-que-%C3%A9-cd2e8d302ae0

https://minerandodados.com.br/one-hot-encoding-como-funciona-python/

https://stackify.dev/928474-scikit-learns-labelbinarizer-vs-onehotencoder

https://stackoverflow.com/questions/58101126/using-scikit-learn-onehotencoder-with-a-pandas-dataframe#:~:text=OneHotEncoder%20Encodes%20categorical%20integer%20features,matrix)%20into%20a%20Pandas%20Series.

https://nelson-ewert-oliveira.medium.com/um-guia-simples-de-como-tratar-vari%C3%A1veis-categ%C3%B3ricas-em-machine-learning-b791a00da805

https://docs.microsoft.com/pt-br/azure/architecture/data-science-process/spark-model-consumption

In [22]:
newdf = pd.DataFrame()

In [23]:
#todas colunas que não são algo numerico, que represente quantidade, sequencia / ordinal ou escala

colunas_onehot = ['nome_curso', 'uf', 'regiao', 'categoria_adm_IES', 'organizacao_academica', 'turno', 'tipo_ensino_medio',
                    'sexo', 'cor_raca', 'cotas', 'trabalha', 'estado_civil', 'moradia', 'ensino_medio']

In [24]:
encoder = LabelBinarizer()

for i in colunas_onehot:
    encoder.fit(enade_df[i])
    transformed = encoder.transform(enade_df[i]) #transformando uma coluna em varias
    ohe_df = pd.DataFrame(transformed) #salvando as novas colunas como dataframe
    ohe_df.rename(columns=lambda x: i+'_'+ str(translate_df[i][x]) , inplace=True) #renomeando as colunas para ter o nome da coluna antiga mais a categoria, ex: nome_curso_BCC
    newdf = pd.concat([newdf, ohe_df], axis=1) #inserindo em um outro dataframe que conterá todas novas colunas após o one-hot encode

In [25]:
colunas_faltantes =  list(set(colunas_categoricas).difference(set(colunas_onehot)))
colunas_faltantes

['avaliacao_curso',
 'recebeu_cota',
 'condicoes_praticas_materiais',
 'plano_ensino',
 'tempo_cursado',
 'renda_familiar',
 'condicoes_salas_estudantes',
 'horas_estudo',
 'range_idade',
 'tempo_ocioso',
 'pessoas_moradia',
 'escolaridade_pai',
 'idade_comeco_graduacao',
 'escolaridade_mae',
 'livros_ano',
 'nt_geral_categoria']

In [26]:
# inserindo as colunas que não passaram pelo one-hot encode
for i in colunas_faltantes:
    newdf[i] = enade_df[i]

In [27]:
newdf

,nome_curso_ADS,nome_curso_BCC,nome_curso_EC,nome_curso_GTI,nome_curso_LCC,nome_curso_RC,nome_curso_SI,uf_AC,uf_AL,uf_AM,...,condicoes_salas_estudantes,horas_estudo,range_idade,tempo_ocioso,pessoas_moradia,escolaridade_pai,idade_comeco_graduacao,escolaridade_mae,livros_ano,nt_geral_categoria
0,0,0,0,0,0,1,0,0,0,0,...,0,1,2,13,3,0,17,0,1,1
1,0,0,0,0,0,1,0,0,0,0,...,1,3,2,15,3,0,20,3,1,1
2,0,0,0,0,0,1,0,0,0,0,...,4,2,0,1,6,0,5,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,4,3,2,12,3,0,17,0,2,1
4,0,0,0,0,0,1,0,0,0,0,...,1,3,0,3,7,0,6,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123487,1,0,0,0,0,0,0,0,0,0,...,4,3,0,1,4,1,4,1,0,4
123488,1,0,0,0,0,0,0,0,0,0,...,1,3,0,5,3,2,9,2,4,1
123489,1,0,0,0,0,0,0,0,0,0,...,4,2,2,9,6,2,20,0,2,1
123490,0,0,0,0,1,0,0,0,0,0,...,2,1,0,6,6,1,9,1,2,4


In [28]:
#pegando somente a coluna nota_geral
copia_df = copia_df[['nota_geral']]

In [29]:
# inserindo a coluna nota_geral no novo dataframe
newdf = pd.concat([newdf, copia_df], axis=1)

In [30]:
newdf.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123492 entries, 0 to 123491
Data columns (total 99 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   nome_curso_ADS                              123492 non-null  int64  
 1   nome_curso_BCC                              123492 non-null  int64  
 2   nome_curso_EC                               123492 non-null  int64  
 3   nome_curso_GTI                              123492 non-null  int64  
 4   nome_curso_LCC                              123492 non-null  int64  
 5   nome_curso_RC                               123492 non-null  int64  
 6   nome_curso_SI                               123492 non-null  int64  
 7   uf_AC                                       123492 non-null  int64  
 8   uf_AL                                       123492 non-null  int64  
 9   uf_AM                                       123492 non-null  int64  
 

In [31]:
newdf

,nome_curso_ADS,nome_curso_BCC,nome_curso_EC,nome_curso_GTI,nome_curso_LCC,nome_curso_RC,nome_curso_SI,uf_AC,uf_AL,uf_AM,...,horas_estudo,range_idade,tempo_ocioso,pessoas_moradia,escolaridade_pai,idade_comeco_graduacao,escolaridade_mae,livros_ano,nt_geral_categoria,nota_geral
0,0,0,0,0,0,1,0,0,0,0,...,1,2,13,3,0,17,0,1,1,0.365023
1,0,0,0,0,0,1,0,0,0,0,...,3,2,15,3,0,20,3,1,1,0.377934
2,0,0,0,0,0,1,0,0,0,0,...,2,0,1,6,0,5,0,0,1,0.382629
3,0,0,0,0,0,1,0,0,0,0,...,3,2,12,3,0,17,0,2,1,0.285211
4,0,0,0,0,0,1,0,0,0,0,...,3,0,3,7,0,6,1,0,3,0.224178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123487,1,0,0,0,0,0,0,0,0,0,...,3,0,1,4,1,4,1,0,4,0.516556
123488,1,0,0,0,0,0,0,0,0,0,...,3,0,5,3,2,9,2,4,1,0.296909
123489,1,0,0,0,0,0,0,0,0,0,...,2,2,9,6,2,20,0,2,1,0.226269
123490,0,0,0,0,1,0,0,0,0,0,...,1,0,6,6,1,9,1,2,4,0.653266


In [32]:
newdf.to_csv('./{}.csv'.format('enade_onehot'),index=False,float_format='%.3f')